In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
# 데이터셋 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10 데이터셋 로드
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 170M/170M [01:18<00:00, 2.16MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  #입력 채널 3, 출력 채널 32, 커널 크기 3x3
        self.pool = nn.MaxPool2d(2, 2)               #풀링 크기 2x2
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) #입력 채널 32, 출력 채널 64, 커널 크기 3x3
        self.fc1 = nn.Linear(64 * 8 * 8, 512)        #완전 연결 층
        self.fc2 = nn.Linear(512, 10)                #출력 층 (10개의 클래스)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)  # 플래튼
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

nn.Conv2d(in_channels, out_channels, kernel_size, padding) : 2차원 합성곱 층을 정의  
nn.MaxPool2d(kernel_size, stride) : 2차원 최대 풀링 층을 정의  

In [4]:
#모델 초기화
model = SimpleCNN()

#손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#모델 학습
for epoch in range(10):  # 10 에포크 동안 학습
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        #기울기 초기화
        optimizer.zero_grad()

        #순전파 + 역전파 + 최적화
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        #손실 출력
        running_loss += loss.item()
        if i % 100 == 99:  # 매 100 미니배치마다 출력
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 100] loss: 2.101
[Epoch 1, Batch 200] loss: 1.727
[Epoch 1, Batch 300] loss: 1.557
[Epoch 1, Batch 400] loss: 1.452
[Epoch 1, Batch 500] loss: 1.377
[Epoch 1, Batch 600] loss: 1.299
[Epoch 1, Batch 700] loss: 1.255
[Epoch 2, Batch 100] loss: 1.167
[Epoch 2, Batch 200] loss: 1.117
[Epoch 2, Batch 300] loss: 1.061
[Epoch 2, Batch 400] loss: 1.059
[Epoch 2, Batch 500] loss: 0.999
[Epoch 2, Batch 600] loss: 1.052
[Epoch 2, Batch 700] loss: 0.980
[Epoch 3, Batch 100] loss: 0.868
[Epoch 3, Batch 200] loss: 0.860
[Epoch 3, Batch 300] loss: 0.836
[Epoch 3, Batch 400] loss: 0.852
[Epoch 3, Batch 500] loss: 0.838
[Epoch 3, Batch 600] loss: 0.845
[Epoch 3, Batch 700] loss: 0.819
[Epoch 4, Batch 100] loss: 0.685
[Epoch 4, Batch 200] loss: 0.695
[Epoch 4, Batch 300] loss: 0.701
[Epoch 4, Batch 400] loss: 0.671
[Epoch 4, Batch 500] loss: 0.694
[Epoch 4, Batch 600] loss: 0.668
[Epoch 4, Batch 700] loss: 0.683
[Epoch 5, Batch 100] loss: 0.518
[Epoch 5, Batch 200] loss: 0.524
[Epoch 5, 

In [5]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')


Accuracy of the network on the 10000 test images: 73.95%
